In [ ]:
from collections import defaultdict
import math
import pandas as pd
import re
from datetime import datetime 
from decimal import Decimal

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
print('numpy: ',np.version.version)
print('pandas: ',pd.__version__)
print('matplotlib: ',matplotlib.__version__)
languages = ['en', 'es', 'eu', 'ca', 'pt', 'gl']

In [ ]:
df = pd.read_csv('../OriginalDataSet/training-tweets.txt', encoding='utf-8', error_bad_lines=False, sep='\t', nrows=10000)
df.columns = ['TweetID', 'UserID', 'Language', "Tweet"]

In [ ]:
_df0 = df[['Language', 'Tweet']].copy()
pattern = re.compile('[ ]')
_df0.head()

In [ ]:
for language in languages:
    exec("%sAlphabets={}" % (language))
    exec ('{0}Size = 0'.format(language))

In [ ]:
start_time = datetime.now() 
trainDict = defaultdict(list)
for index, row in _df0.iterrows():
    sentence = ''
    tweet = row['Tweet']
    language = row['Language']
    for letter in tweet:
        if letter.isalpha() or pattern.match(letter):
            exec('if \'{0}\' not in {1}Alphabets.keys():\n\
                     {2}Alphabets[letter] = {3}Size\n\
                     {4}Size += 1'.format(letter, language, language, language, language))
            sentence = sentence + letter
    trainDict[row['Language']].append(sentence)
print('Reading time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

In [ ]:
print(enSize)
print(enAlphabets)
print(esSize)
print(esAlphabets)
print(euSize)
print(euAlphabets)
print(caSize)
print(caAlphabets)
print(ptSize)
print(ptAlphabets)
print(glSize)
print(glAlphabets)
print(trainDict)

In [ ]:
for language in languages:
    exec("%sModel=np.zeros(shape=(%sSize+1))" %(language, language))
#     exec("%sCount=0" %(language))
#     exec("%sEachWordCount=dict.fromkeys(%sAlphabets, 0)" %(language, language)) #needed for bigrams

In [ ]:
print('EN',enModel, enModel.shape)
print('ES',esModel, esModel.shape)
print('EU',euModel, euModel.shape)
print('CA',caModel, caModel.shape)
print('PT',ptModel, ptModel.shape)
print('gl',glModel, glModel.shape)

In [ ]:
# a = dict.fromkeys(a, 0)

#### For Unigrams

In [ ]:
start_time = datetime.now()
delta = 0.5
for language, tweets in trainDict.items():
    for tweet in tweets:
        for i in range(len(tweet)-1):
            first = tweet[i]
            exec('index = %sAlphabets[first]'%(language)) #index = esAlphabets[first] #get index of the character from the language dictionary
#             exec('np.add.at(%sModel, [index], 1)'%(language)) #np.add.at(esModel, [index], 1) #increment that index in the language model
            exec('%sModel[index] += 1'%(language))
for language in languages:
    exec('%sModel = np.add(%sModel, delta)'%(language, language)) #this is where smoothing happens
    exec('%sModel = %sModel/%sModel.sum()'%(language, language, language)) #divide all the values by the sum of the row
    exec('%sModel = np.log10(%sModel)'%(language, language))
print('Training time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

In [ ]:
print('EN',enModel, enModel.shape)
print('ES',esModel, esModel.shape)
print('EU',euModel, euModel.shape)
print('CA',caModel, caModel.shape)
print('PT',ptModel, ptModel.shape)
print('gl',glModel, glModel.shape)

In [ ]:
# glSum = glModel.sum()
# ptSum = ptModel.sum()
# caSum = caModel.sum()
# euSum = euModel.sum()
# esSum = esModel.sum()
# enSum = enModel.sum()

In [ ]:
# print('enSum', enSum ,'\nesSum', esSum ,'\neuSum', euSum ,'\ncaSum', caSum ,'\nptSum', ptSum, '\nglSum', glSum)

In [ ]:
# print(enAlphabets.values())
# print(enModel)
# print(type(list(enAlphabets.values())))
# npa = np.asarray(someListOfLists, dtype=np.float32)
x = np.asarray(list(enAlphabets.values()))
x = np.append(x, (x.shape[0]))
y = enModel
print('x:\n', x)
print('y:\n', y)

fig = plt.figure()
fig.set_size_inches(15, 3)
area = (10 * np.random.rand(x.shape[0]))**2
colors = np.random.rand(x.shape[0])
plt.title('SUQ MADIQ')
plt.xlabel('Alphabets')
plt.ylabel('Probability of alphabet')
plt.scatter(x, y, s=area, c=colors, alpha=0.7)
plt.xticks(x, list(enAlphabets.keys()))
plt.tick_params(axis='x', which='minor', labelsize=10, width = 5)
# plt.grid()
# plt.show()

In [ ]:
df = pd.read_csv('../OriginalDataSet/test-tweets-given.txt', encoding='utf-8', error_bad_lines=False, sep='\t', nrows=100)
df.columns = ['TweetID', 'UserID', 'Language', "Tweet"]
_df0 = df[['TweetID', 'Language', 'Tweet']].copy()
# pattern = re.compile('[a-z ]')
_df0.head()

In [ ]:
probability = {}
for index, row in _df0.iterrows():
    for language in languages:
        exec("%sProb=math.log10(1/6)" % (language))
    tweetID = row['TweetID']
    langTweet = row['Language']
    tweet = row['Tweet']
    
    for i in range(len(tweet)-2):
        first = tweet[i]
        for lang in languages:
            exec('if (first not in {0}Alphabets.keys()):\n\
    prob = {1}Model[-1]\n\
else:\n\
    index = {2}Alphabets[first]\n\
    prob = {3}Model[index]\n\
{4}Prob = prob + {5}Prob\n'.format(lang, lang, lang, lang, lang, lang)) #, lang, lang, lang, lang, lang, lang, lang, lang, lang))
    for langu in languages:
        exec("probability['%s'] = %sProb"%(langu, langu))
#     probability[lang] = enProb
#     print(probability)
#     print('tweet: ', tweet,'\nenProb:', enProb, '\neuProb:', euProb, '\nesProb:', esProb, '\ncaProb:', caProb
#     ,'\nptProb:', ptProb, '\nglProb:', glProb)
    result = max(probability, key=probability.get)
    print(tweetID, '  ', result, '  ', '%.2E' % Decimal(probability[result]), '  ', langTweet, 'correct' if (langTweet==result) else 'wrong')

In [ ]:
a = 'if not self.pattern.match(first):\n\
    prob = 0\n\
elif (first not in self.{lang}Alphabets.keys()):\n\
    prob = self.{lang}Model[-1]\n\
else:\n\
    index = self.{lang}Alphabets[first]\n\
    prob = self.{lang}Model[index]\n\
{lang}Prob = prob + {lang}Prob\n'.format(lang='en')
print(a)